# Date Translation with Attention

In [25]:
from tensorflow.keras.layers import Bidirectional, Concatenate, Dot, Input, LSTM
from tensorflow.keras.layers import RepeatVector, Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
import numpy as np

from nmt_utils import load_dataset, preprocess_data, softmax, string_to_int

## 1. Dataset

In [2]:
# Dataset là các cặp human readable date và machine readable date. Input sẽ là một date đc viết theo nhiều dạng khác nhau
# (VD "the 29th of August 1958", "03/30/1968", "24 JUNE 1987"), ta cần dịch nó sang machine-readable format YYYY-MM-DD. 
m = 10000
dataset, human_vocab, machine_vocab, inv_machine_vocab = load_dataset(m)
# dataset = [('9 may 1998', '1998-05-09'), ('10.11.19', '2019-11-10'), ...], len=10000
# human_vocab = {' ': 0, '.': 1, '/': 2, ..., <unk>': 35, '<pad>': 36}, len=37
# machine_vocab = {'-': 0, '0': 1, '1': 2, '2': 3, '3': 4, '4': 5, '5': 6, '6': 7, '7': 8, '8': 9, '9': 10}, len=11
# inv_machine_vocab = {0: '-', 1: '0', 2: '1', 3: '2', 4: '3', 5: '4', 6: '5', 7: '6', 8: '7', 9: '8', 10: '9'}, len=11

100%|██████████| 10000/10000 [00:00<00:00, 34394.26it/s]


In [6]:
# Ở đây T_x=30 (số lượng characters tối đa cho phép trong human date), T_y=10 (số lượng characters trong format YYYY-MM-DD)
Tx = 30
Ty = 10
X, Y, Xoh, Yoh = preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty)
# X.shape=(m, Tx)=(10000, 30), VD X[0]=[12  0 24 13 34  0  4 12 12 11 36 36 ... 36] là các indices từ human_vocab của text '9 may 1998'
# Y.shape=(m, Ty)=(10000, 10), VD Y[0]=[2 10 10  9  0  1  6  0  1 10] là các indices từ machine_vocab của text '1998-05-09'
# Xoh.shape=(m, Tx, len(human_vocab))=(10000, 30, 37), là one-hot vector từ X, VD X[0, 0]=12 thì Xoh[0, 0, 12]=1
# Yoh.shape=(m, Ty, len(machine_vocâb))=(10000, 10, 11), là one-hot vector từ Y, VD Y[0, 0]=2 thì Yoh[0, 0, 2]=1

X.shape: (10000, 30)
Y.shape: (10000, 10)
Xoh.shape: (10000, 30, 37)
Yoh.shape: (10000, 10, 11)


## 2. Model

<img src="images/attn_model.png" height=600/>
<img src="images/attn_mechanism.png" height=600/>

#### one_step_attention
Xét tại một time step t và chỉ xét một single sample. Inputs của __one_step_attention__ là:
- Tất cả hidden state $a=[a^{<1>},a^{<2>}, ..., a^{<T_x>}]$ từ pre-attention Bi-LSTM, có shape $(T_x, 2n_a)=(30, 64)$. $a^{<t>}$ là một vector dài $2n_a=64$.
- Previous hidden state $s^{<t-1>}$ từ post-attention LSTM, là một vecctor dài $n_s=64$.

Các bước trong __one_step_attention__:
1. Concatenate từng $a^{<t'>}$ với $s^{<t-1>}$ (dùng t' để phân biệt với time step đang xét là t), KQ có là vector dài $n_s+2n_a=128$
2. Cho vector này đi qua FC layer, output chỉ có 1 unit chính là $\alpha^{<t, t'>}$
3. Sau khi thực hiện hai bước trên với tất cả $T_x$ cái $a^{<t'>}$, ta sẽ thu đc $T_x$ cái $\alpha^{<t, t'>}$. Giờ ính tổng:
   $$context^{<t>} = \sum_{t' = 1}^{T_x} \alpha^{<t,t'>}a^{<t'>}$$ 
   Ở đây mỗi $\alpha^{<t, t'>}$ là một số, còn mỗi $a^{<t'>}$ là một vector dài $2n_a=64$, KQ thu đc vector $context^{<t>}$ dài $2n_a=64$ 

In [14]:
# Defined shared layers as global variables
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation(softmax, name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor = Dot(axes = 1)

def one_step_attention(a, s_prev): # Trong một khối Attention
    # 2.1. Lặp lại s^{t-1} Tx lần để nó có shape (batch_size, Tx, ns)=(100, 30, 64), rồi concatenate nó với a, giờ sẽ có shape (batch_size, Tx, ns+2na)=(100, 30, 128)
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a" (≈ 1 line)
    s_prev = repeator(s_prev)
    # Use concatenator to concatenate a and s_prev on the last axis (≈ 1 line)
    # For grading purposes, please list 'a' first and 's_prev' second, in this order.
    concat = concatenator([a,s_prev])

    # 2.2. Cho nó đi qua một FCN (2 lớp Dense) rồi đến softmax để tính alphas, giờ có shape (batch_size, Tx, 1)=(100, 30, 1) (1 là số units lớp Dense cuối)
    # Use densor1 to propagate concat through a small fully-connected neural network to compute the "intermediate energies" variable e. (≈1 lines)
    e = densor1(concat)
    # Use densor2 to propagate e through a small fully-connected neural network to compute the "energies" variable energies. (≈1 lines)
    energies = densor2(e)
    # Use "activator" on "energies" to compute the attention weights "alphas" (≈ 1 line)
    alphas = activator(energies)

    # 2.3. Tính context theo công thức bằng cách tính tổng cách tích element-wise alphas với a theo các time step Tx, KQ có shape (batch_size, 30, 2na)=(100, 30, 64)
    #      Ở đây context[:, t, :]= Tổng các alphas[:, t, 0] (là một số) nhân với a[:, t, :] (là một vector)
    # Use dotor together with "alphas" and "a", in this order, to compute the context vector to be given to the next (post-attention) LSTM-cell (≈ 1 line)
    context = dotor([alphas,a])
    
    return context

In [17]:
n_a = 32 # number of units for the pre-attention, bi-directional LSTM's hidden state 'a'
n_s = 64 # number of units for the post-attention LSTM's hidden state "s"

# Please note, this is the post attention LSTM cell.  
post_activation_LSTM_cell = LSTM(n_s, return_state = True) # Please do not modify this global variable.
output_layer = Dense(len(machine_vocab), activation=softmax)

In [15]:
def modelf(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size): # Sẽ dùng batch_size=100; Xét một batch của X có shape (batch_size, Tx, 37)=(100, 30, 37)
    # Define the inputs of your model with a shape (Tx,)
    # Define s0 (initial hidden state) and c0 (initial cell state)
    # for the decoder LSTM with shape (n_s,)
    X = Input(shape=(Tx, human_vocab_size))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    # Initialize empty list of outputs
    outputs = []
    
    # 1. Đi qua một lớp Bidirectional LSTM, lấy ra toàn bộ hidden states, có shape (batch_size, 30, 2na)=(10, 30, 64)
    # Step 1: Define your pre-attention Bi-LSTM. (≈ 1 line)
    a = Bidirectional(LSTM(n_a, return_sequences=True))(X)
    
    # Step 2: Iterate for Ty steps
    for t in range(Ty): # Tại một time step t
        # 2. Đưa s^{t-1} (previous hidden state của post LSTM cell), có shape (batch_size, ns)=(100, 64) và a vào một khối Attention (Xem tiếp khối Attention)
        #    Output là context^t có shape (batch_size, 30, 2na)=(100, 30, 64)
        # Step 2.A: Perform one step of the attention mechanism to get back the context vector at step t (≈ 1 line)
        context = one_step_attention(a, s)
        
        # 3. Cho context đi qua một LSTM cell khác, lấy ra last hidden state s^t
        # Step 2.B: Apply the post-attention LSTM cell to the "context" vector.
        # Don't forget to pass: initial_state = [hidden state, cell state] (≈ 1 line)
        _, s, c = post_activation_LSTM_cell(context, initial_state=[s, c])
        
        # 4. Cho s^t đi qua một lớp Dense có số units là len(machine_vocab)=11, thể hiện XS một character trong machine_vocab đc chọn
        #    out là y^t, có shape (batch_size, len(machine_vocab))=(100, 11)
        # Step 2.C: Apply Dense layer to the hidden state output of the post-attention LSTM (≈ 1 line)
        out = output_layer(s)
        
        # Step 2.D: Append "out" to the "outputs" list (≈ 1 line)
        outputs.append(out)
    
    # Step 3: Create model instance taking three inputs and returning the list of outputs. (≈ 1 line)
    model = Model(inputs=[X, s0, c0],outputs=outputs)
    
    return model

In [19]:
model = modelf(Tx, Ty, n_a, n_s, len(human_vocab), len(machine_vocab))
# model.summary()

In [23]:
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
outputs = list(Yoh.swapaxes(0,1))

opt = Adam(learning_rate=0.005, beta_1=0.9, beta_2=0.999, weight_decay=0.01)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy']*10)
model.fit([Xoh, s0, c0], outputs, epochs=1, batch_size=100, verbose=1)

100/100 ━━━━━━━━━━━━━━━━━━━━ 11s 28ms/step - dense_2_accuracy: 0.9423 - dense_2_accuracy_1: 0.9423 - dense_2_accuracy_2: 0.4339 - dense_2_accuracy_3: 0.1583 - dense_2_accuracy_4: 0.9787 - dense_2_accuracy_5: 0.8168 - dense_2_accuracy_6: 0.2914 - dense_2_accuracy_7: 0.9782 - dense_2_accuracy_8: 0.4967 - dense_2_accuracy_9: 0.1716 - dense_2_loss: 2.2404 - loss: 9.9748


In [24]:
model.load_weights('models/model.h5')

In [40]:
EXAMPLES = ['3 May 1979', '5 April 09', '21th of August 2016', 'Tue 10 Jul 2007', 'Saturday May 9 2018', 'March 3 2001', 'March 3rd 2001', '1 March 2001']
s00 = np.zeros((1, n_s))
c00 = np.zeros((1, n_s))
for example in EXAMPLES: # e.g. example='3 May 1979'
    # Chuyển sang indices từ human vocab
    source = string_to_int(example, Tx, human_vocab) # [6, 0, 24, 13, ..., 36, 36], len=Tx=30

    # Chuyển sang one-hot
    source = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), source)))
    # shape=(30, 37)
    # [[0. 0. 0. ... 0. 0. 0.]
    #  [1. 0. 0. ... 0. 0. 0.]
    #  ...
    # ]
    source = np.expand_dims(source, axis=0) # shape=(1, 30, 37)
    
    prediction = model.predict([source, s00, c00]) # Là một list dài Ty=10, mỗi element có shape (1, 11)
    prediction = np.argmax(prediction, axis = -1) # Là một list dài Ty=10, mỗi element có shape (1,)

    output = [inv_machine_vocab[int(i)] for i in prediction]
    print("source:", example)
    print("output:", ''.join(output),"\n")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
source: 3 May 1979
output: 1979-05-33 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
source: 5 April 09
output: 2009-04-05 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
source: 21th of August 2016
output: 2016-08-20 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
source: Tue 10 Jul 2007
output: 2007-07-10 



/tmp/ipykernel_94794/3715588122.py:20: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  output = [inv_machine_vocab[int(i)] for i in prediction]


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
source: Saturday May 9 2018
output: 2018-05-09 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
source: March 3 2001
output: 2001-03-03 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
source: March 3rd 2001
output: 2001-03-03 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
source: 1 March 2001
output: 2001-03-01 



### Notes
* Trong post LSTM, một cell tại time $t$ ko dùng prediction từ time $t-1$ làm input, mà chỉ lấy hidden state và cell state. Ta thiết kế model kiểu này vì ko giống như language generation, ko có strong dependency giữa các characters trong format YYYY-MM-DD.
* Xét VD dịch "Saturday 9 May 2018" sang "2018-05-09". Khi visualize các $\alpha^{<t, t'>}$, ta sẽ thấy các phần của output tập trung vào các phần của input ntn.

  <img src="images/date_attention.png" height=400/>
    
    Ko có phần nào của output tập trung vào phần “Saturday” trong input. Các phần “9” và “May” đã đc dịch đúng sang 09 và 05. Phần year chỉ tập trung vào “18” trong input để dịch sang 2018.

